In [1]:
from flask import Flask, render_template, request
import pandas as pd
import io
app = Flask(__name__)

In [2]:
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Get the uploaded file
        uploaded_file = request.files['file']

        if uploaded_file.filename != '':
            # Read the CSV file data using pandas
            data = pd.read_csv(uploaded_file)
            
           
            data_html = data.to_html()
            return render_template('index.html', table=data_html)
            
    return render_template('index.html', table='')


In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.0.2.15:5000
Press CTRL+C to quit
10.0.2.2 - - [09/Aug/2023 11:22:32] "GET / HTTP/1.1" 200 -
10.0.2.2 - - [09/Aug/2023 11:24:26] "POST / HTTP/1.1" 200 -
10.0.2.2 - - [09/Aug/2023 11:25:39] "POST / HTTP/1.1" 200 -
